### POS tagging with maximum entropy models (10 pts)

In this task you will build a maximum entropy model for part-of-speech tagging. As the name suggests, our problem is all about converting a sequence of words into a sequence of part-of-speech tags. 
<img src=https://i.stack.imgur.com/6pdIT.png width=320>


__Your man goal:__ implement the model from [the article you're given](W96-0213.pdf).

Unlike previous tasks, this one gives you greater degree of freedom and less automated tests. We provide you with programming interface but nothing more.

__A piece of advice:__ there's a lot of objects happening here. If you don't understand why some object is needed, find `def train` function and see how everything is linked together.


In [2]:
import collections
import numpy as np
import os
import pprint
import pickle
import time

### Part I: reading input data

In [3]:
# Data types:
# Word: str
# Sentence: list of str
TaggedWord = collections.namedtuple('TaggedWord', ['text', 'tag'])
# TaggedSentence: list of TaggedWord
# Tags: list of TaggedWord
# TagLattice: list of Tags

def read_tagged_sentences(path):
    """
    Read tagged sentences from CoNLL-U file and return array of TaggedSentence (array of lists of TaggedWord).
    """
    tagged_sentences = []
    with open(path) as fin:
        sentence = []
        for line in fin:
            line = line.strip()
            if line.startswith("#"):
                continue
            if not line:
                if sentence:
                    tagged_sentences.append(sentence)
                    sentence = []
                continue
            tokens = line.split("\t")
            sentence.append(TaggedWord(text=tokens[1], tag=tokens[3]))
    return tagged_sentences
            
            
            

def write_tagged_sentence(tagged_sentence, f):
    """
    Write tagged sentence in CoNLL-U format to file-like object f.
    """
    for idx, word in enumerate(tagged_sentence):
        line = [str(idx), word.text, "_", word.tag] + ["_"] * 6
        f.write("\t".join(line) + "\n")
        

def read_tags(path):
    """
    Read a list of possible tags from file and return the list.
    """
    tags = []
    with open(path) as fin:
        for line in fin:
            tags.append(line.strip())
    return tags

### Part II: evaluation

We want you to estimate tagging quality by a simple accuracy: a fraction of tag predictions that turned out to be correct - averaged over the entire training corpora.

In [4]:
# Data types:
TaggingQuality = collections.namedtuple('TaggingQuality', ['acc'])

def tagging_quality(ref, out):
    """
    Compute tagging quality and return TaggingQuality object.
    """
    nwords = 0
    ncorrect = 0
    import itertools
    for ref_sentence, out_sentence in itertools.zip_longest(ref, out):
        for ref_word, out_word in itertools.zip_longest(ref_sentence, out_sentence):         
            if ref_word and out_word and ref_word.tag == out_word.tag:
                ncorrect += 1
            nwords += 1
    return ncorrect / nwords

### Part III: Value and Update

In order to implement two interlinked data structures: 
* __Value__ - a class that holds POS tagger's parameters. Basically an array of numbers
* __Update__ - a class that stores updates for Value

In [5]:
class Value:
    def __init__(self, n, random=False):
        """
        Dense object that holds parameters.
        :param n: array length
        """
        self.values = np.random.normal(size=n) if random else np.zeros(n)

    def dot(self, update):
        result = 0
        for pos, value in zip(update.positions, update.values):
            result += self.values[pos] * value
        return result

    def assign(self, other):
        """
        self = other
        other is Value.
        """
        self.values = other.values

    def assign_mul(self, coeff):
        """
        self = self * coeff
        coeff is float.
        """
        self.values *= coeff

    def assign_madd(self, x, coeff):
        """
        self = self + x * coeff
        x can be either Value or Update.
        coeff is float.
        """
        x.assign_mul(coeff)
        if isinstance(x, Value):
            self.assign(x)
        elif isinstance(x, Update):
            for pos, value in zip(x.positions, x.values):
                self.values[pos] += value
        else:
            raise ValueError("x can be either Value or Update.")


class Update:
    """
    Sparse object that holds an update of parameters.
    """

    def __init__(self, positions=None, values=None):
        """
        positions: array of int
        values: array of float
        """
        self.positions = positions if positions is not None else np.array([])
        self.values = values if values is not None else np.array([])

    def assign_mul(self, coeff):
        """
        self = self * coeff
        coeff: float
        """
        for idx, _ in enumerate(self.values):
            self.values[idx] *= coeff

    def assign_madd(self, update, coeff):
        """
        self = self + update * coeff
        coeff: float
        """
        update.assign_mul(coeff)
        pos_to_value = collections.defaultdict(float)
        for pos, value in zip(self.positions, self.values):
            pos_to_value[pos] += value
        for pos, value in zip(update.positions, update.values):
            pos_to_value[pos] += value
        self.positions = []
        self.values = []
        for pos, value in pos_to_value.items():
            if value:
                self.positions.append(pos)
                self.values.append(value)
        self.positions = np.array(self.positions)
        self.values = np.array(self.values)

### Part IV: Maximum Entropy POS Tagger
_step 1 - draw an oval; step 2 - draw the rest of the owl (c)_

In this secion you will implement a simple linear model to predict POS tags.
Make sure you [read the article](W96-0213.pdf) before you proceed.

In [6]:
# Data Types:
Features = Update
Hypo = collections.namedtuple('Hypo', ['prev', 'pos', 'tagged_word', 'score'])
# prev: previous Hypo
# pos: position of word (0-based)
# tagged_word: tagging of source_sentence[pos]
# score: sum of scores over edges

TaggerParams = collections.namedtuple('FeatureParams', [
    'src_window',
    'dst_order',
    'max_suffix',
    'beam_size',
    'nparams'
    ])

import cityhash
def h(x):
    """
    Compute CityHash of any object.
    Can be used to construct features.
    """
    return cityhash.CityHash64(repr(x))

/Users/nastya/.virtualenvs/nlp_pypy3/lib-python/3/importlib/_bootstrap.py:223: UserWarning: builtins.type size changed, may indicate binary incompatibility. Expected 872, got 416
  return f(*args, **kwds)


In [7]:
class LinearModel:
    """
    A thing that computes score and gradient for given features.
    """

    def __init__(self, n):
        self._params = Value(n)

    def params(self):
        return self._params

    def score(self, features):
        """
        features: Update
        """
        return self._params.dot(features)

    def gradient(self, features, score):
        return features

In [8]:
class FeatureComputer:
    def __init__(self, tagger_params, source_sentence):
        self.tagger_params = tagger_params
        self.source_sentence = source_sentence
        self.sent_len = len(source_sentence)

    def compute_features(self, hypo):
        """
        Compute features for a given Hypo and return Update.
        """
        tagged_word = hypo.tagged_word
        positions = [h(f"word={tagged_word.text},tag={tagged_word.tag}")]
        for i in range(1, self.tagger_params.src_window + 1):
            word_pos_minus_i = self.source_sentence[hypo.pos - i].text if hypo.pos - i >= 0 else "None"
            word_pos_plus_i = (self.source_sentence[hypo.pos + i].text if hypo.pos + i < self.sent_len else "None")
            feat_str_minus_i = f"word_{-i}='{word_pos_minus_i}',tag='{tagged_word.tag}'"
            feat_str_plus_i = f"word_{i}='{word_pos_plus_i}',tag='{tagged_word.tag}'"
            # print(feat_str_minus_i)
            # print(feat_str_plus_i)
            positions.append(h(feat_str_minus_i))
            positions.append(h(feat_str_plus_i))
        tag_sequence = []
        current_hypo = hypo.prev if hypo else None
        for i in range(1, self.tagger_params.dst_order):
            tag_sequence.append(current_hypo.tagged_word.tag if current_hypo else "None")
            tag_seq_str = " ".join(tag_sequence)
            # print(tag_seq_str)
            positions.append(h(
                f"tag_seq_{i}='{tag_seq_str}',tag='{tagged_word.tag}'"))
            current_hypo = current_hypo.prev if current_hypo else None
        for i in range(1, self.tagger_params.max_suffix + 1):
            suffix_feat = f"suff='{tagged_word.text[-i:]}',tag='{tagged_word.tag}'"
            prefix_feat = f"pref='{tagged_word.text[:i]}',tag='{tagged_word.tag}'"
            # print(suffix_feat)
            # print(prefix_feat)
            positions.append(h(prefix_feat))
            positions.append(h(suffix_feat))
        positions = list(map(lambda x: x % self.tagger_params.nparams, positions))
        return Update(positions=np.array(positions), values=np.ones(len(positions)))

In [9]:
sentence = read_tagged_sentences("data/en-ud-debug.conllu")[0]
print(sentence)
params = TaggerParams(src_window=2, dst_order=3, max_suffix=4, nparams=2 ** 22, beam_size=1)
computer = FeatureComputer(params, sentence)
prev_hypo = Hypo(prev = None, pos=2, tagged_word=sentence[2], score=228)
hypo = Hypo(prev=prev_hypo, pos=3, tagged_word=sentence[3], score=100500)
print(computer.compute_features(hypo).positions)

[TaggedWord(text='From', tag='ADP'), TaggedWord(text='the', tag='DET'), TaggedWord(text='AP', tag='PROPN'), TaggedWord(text='comes', tag='VERB'), TaggedWord(text='this', tag='DET'), TaggedWord(text='story', tag='NOUN'), TaggedWord(text=':', tag='PUNCT')]
[3327057   45182 1862397 1334518 3828493 3424278 1835926  799099   82873
 1616455 2131331 2688935  498780 3750176 1722868]


### Part V: Beam search

We can find the most likely tagging approximately using Beam Search. As everything else, it comes with a separate interface.

In [10]:
class BeamSearchTask:
    """
    An abstract beam search task. Can be used with beam_search() generic 
    function.
    """

    def __init__(self, tagger_params, source_sentence, model, tags):
        self.tagger_params = tagger_params
        self.source_sentence = source_sentence
        self.model = model
        self.tags = tags
        self.feat_computer = FeatureComputer(tagger_params, source_sentence)

    def total_num_steps(self):
        """
        Number of hypotheses between beginning and end (number of words in
        the sentence).
        """
        return len(self.source_sentence)

    def beam_size(self):
        return self.tagger_params.beam_size
            
    def expand(self, hypo):
        """
        Given Hypo, return a list of its possible expansions.
        'hypo' might be None -- return a list of initial hypos then.

        Compute hypotheses' scores inside this function!
        """
        hypos = []
        if hypo is not None and hypo.pos == self.total_num_steps() - 1:
            return hypos
        for tag in self.tags:
            new_pos = hypo.pos + 1 if hypo is not None else 0
            prev_score = hypo.score if hypo is not None else 0
            hypo_to_score = Hypo(prev=hypo, pos=new_pos,
                                 tagged_word=TaggedWord(text=self.source_sentence[new_pos].text,
                                 tag=tag), score=0)
            new_hypo_score = self.model.score(self.feat_computer.compute_features(hypo_to_score))
            hypos.append(Hypo(prev=hypo, pos=new_pos,
                              tagged_word=TaggedWord(text=self.source_sentence[new_pos].text,
                              tag=tag),
                              score=new_hypo_score + prev_score))
        return hypos

    def recombo_hash(self, hypo):
        """
        If two hypos have the same recombination hashes, they can be collapsed
        together, leaving only the hypothesis with a better score.
        """
        tag_sequence = []
        current_hypo = hypo
        for i in range(self.tagger_params.dst_order):
            tag_sequence.append(current_hypo.tagged_word.tag if current_hypo else "None")
        return h(tag_sequence)
        


def beam_search(beam_search_task):
    """
    Return list of stacks.
    Each stack contains several hypos, sorted by score in descending 
    order (i.e. better hypos first).
    """
    all_possible_hypos = beam_search_task.expand(None)
    prev_stack = sorted(all_possible_hypos,
                        key=lambda hypo: -hypo.score)[:beam_search_task.beam_size()]
    stacks = [prev_stack]
    for i in range(beam_search_task.total_num_steps() - 1):
        proposed_hypos = []
        for hypo in prev_stack:
            proposed_hypos.extend(beam_search_task.expand(hypo))
        hash_to_hypos = collections.defaultdict(list)
        for hypo_ in proposed_hypos:
            hash_to_hypos[beam_search_task.recombo_hash(hypo_)].append(hypo_)
        best_hypos_without_copies = []
        for _, equal_hash_hypos in hash_to_hypos.items():
            best_hypo = equal_hash_hypos[0]
            for hypo_ in equal_hash_hypos:
                if best_hypo.score < hypo_.score:
                    best_hypo = hypo_
            best_hypos_without_copies.append(best_hypo)
        prev_stack = sorted(best_hypos_without_copies,
                            key=lambda hypo: -hypo.score)[:beam_search_task.beam_size()]
        stacks.append(prev_stack)
    return stacks

In [11]:
def tag_sentences(dataset, model, tags, tagger_params):
    """
    Main predict function.
    Tags all sentences in dataset. Dataset is a list of TaggedSentence; while 
    tagging, ignore existing tags.
    """
    tagged_dataset = []
    for idx, sentence in enumerate(dataset):     
        beam_search_task = BeamSearchTask(tagger_params, sentence, model, tags)
        hypos = []
        stacks = beam_search(beam_search_task)
        best_hypo = stacks[-1][0]
        current_hypo = best_hypo
        while current_hypo is not None:
            hypos.append(current_hypo)
            current_hypo = current_hypo.prev
        tagged_sentence = []
        for hypo in hypos[::-1]:
            tagged_sentence.append(hypo.tagged_word)
        tagged_dataset.append(tagged_sentence)
        if idx == 0:
            print(tagged_sentence)
    return tagged_dataset

### Part VI: Optimization objective and algorithm

Once we defined our model and inference algorithm, we can define an optimization task: an object that computes loss function and its gradients w.r.t. model parameters.

In [12]:
class OptimizationTask:
    """
    Optimization task that can be used with sgd().
    """

    def params(self):
        """
        Parameters which are optimized in this optimization task.
        Return Value.
        """
        raise NotImplementedError()

    def loss_and_gradient(self, golden_sentence):
        """
        Return (loss, gradient) on a specific example.

        loss: float
        gradient: Update
        """
        raise NotImplementedError()


class UnstructuredPerceptronOptimizationTask(OptimizationTask):
    def __init__(self, tagger_params, tags):
        self.model = LinearModel(tagger_params.nparams)
        self.tagger_params = TaggerParams(src_window=tagger_params.src_window,
                                          dst_order=tagger_params.dst_order,
                                          max_suffix=tagger_params.max_suffix,
                                          beam_size=1,
                                          nparams=tagger_params.nparams)
        self.tags = tags

    def params(self):
        return self.model.params()

    def loss_and_gradient(self, golden_sentence):
        beam_search_task = BeamSearchTask(
        self.tagger_params, 
        golden_sentence,
        # [golden_tagged_word.text for golden_tagged_word in golden_sentence], 
        self.model, 
        self.tags
        )
        stacks = beam_search(beam_search_task)
        golden_hypos = []
        golden_hypo = None
        feature_computer = FeatureComputer(self.tagger_params, golden_sentence)
        for i in range(len(golden_sentence)):
            new_golden_hypo = Hypo(prev=golden_hypo, pos=i,
                                   tagged_word=golden_sentence[i],
                                   score=0)
            new_hypo_score = self.model.score(feature_computer.compute_features(new_golden_hypo))
            new_golden_hypo = Hypo(prev=golden_hypo, pos=i,
                                   tagged_word=golden_sentence[i],
                                   score=new_hypo_score)
            golden_hypo = new_golden_hypo
            golden_hypos.append(golden_hypo)
        golden_head = golden_hypos[-1]
        rival_head = stacks[-1][0]
        grad = Update()
        loss = 0
        while golden_head and rival_head:
            rival_features = feature_computer.compute_features(rival_head)
            grad.assign_madd(self.model.gradient(rival_features, score=None), 1)

            golden_features = feature_computer.compute_features(golden_head)
            grad.assign_madd(self.model.gradient(golden_features, score=None), -1)
            loss += rival_head.score - golden_head.score
            golden_head = golden_head.prev
            rival_head = rival_head.prev
        return loss, grad
        


class StructuredPerceptronOptimizationTask(OptimizationTask):
    def __init__(self, tagger_params, tags):
        self.tagger_params = tagger_params
        self.model = LinearModel(self.tagger_params.nparams)
        self.tags = tags

    def params(self):
        return self.model.params()

    def loss_and_gradient(self, golden_sentence):
        # Do beam search.
        beam_search_task = BeamSearchTask(
            self.tagger_params, 
            golden_sentence,
            # [golden_tagged_word.text for golden_tagged_word in golden_sentence], 
            self.model, 
            self.tags
            )
        stacks = beam_search(beam_search_task)

        # Compute chain of golden hypos (and their scores!).
        golden_hypos = []
        golden_hypo = None
        feature_computer = FeatureComputer(self.tagger_params, golden_sentence)
        for i in range(len(golden_sentence)):
            new_golden_hypo = Hypo(prev=golden_hypo, pos=i,
                                   tagged_word=golden_sentence[i],
                                   score=0)
            new_hypo_score = self.model.score(feature_computer.compute_features(new_golden_hypo))
            new_golden_hypo = Hypo(prev=golden_hypo, pos=i,
                                   tagged_word=golden_sentence[i],
                                   score=new_hypo_score)
            golden_hypo = new_golden_hypo
            golden_hypos.append(golden_hypo)

        # Find where to update.
        golden_head = None
        rival_head = None
        for idx, hypo in enumerate(golden_hypos):
            if hypo.score < stacks[idx][-1].score:
                golden_head = hypo
                rival_head = stacks[idx][0]
        if golden_head is None and rival_head is None:
            golden_head = golden_hypos[-1] 
            rival_head = stacks[-1][0]
        # Compute gradient.
        grad = Update()
        loss = 0
        while golden_head and rival_head:
            rival_features = feature_computer.compute_features(rival_head)
            grad.assign_madd(self.model.gradient(rival_features, score=None), 1)

            golden_features = feature_computer.compute_features(golden_head)
            grad.assign_madd(self.model.gradient(golden_features, score=None), -1)
            loss += rival_head.score - golden_head.score
            golden_head = golden_head.prev
            rival_head = rival_head.prev

        return loss, grad


### Part VII: optimizer

By this point we can define a model with parameters $\theta$ and a problem that computes gradients $ \partial L \over \partial \theta $ w.r.t. model parameters.

Optimization is performed by gradient descent: $ \theta := \theta - \alpha {\partial L \over \partial \theta} $

In order to speed up training, we use stochastic gradient descent that operates on minibatches of data.

In [13]:
SGDParams = collections.namedtuple('SGDParams', [
    'epochs',
    'learning_rate',
    'minibatch_size',
    'average' # bool or int
    ])


def make_batches(dataset, minibatch_size):
    """
    Make list of batches from a list of examples.
    """
    batches = []
    batch = []
    shuffled_dataset = np.random.permutation(dataset)
    for sentence in shuffled_dataset:
        batch.append(sentence)
        if len(batch) % minibatch_size == 0:
            batches.append(batch)
            batch = []
    if len(batch) > 0:
        batches.append(batch)
    return batches


def sgd(sgd_params, optimization_task, dataset, after_each_epoch_fn):
    """
    Run (averaged) SGD on a generic optimization task. Modify optimization
    task's parameters.

    After each epoch (and also before and after the whole training),
    run after_each_epoch_fn().
    """
    after_each_epoch_fn()
    all_params_avg = optimization_task.params()
    for epoch in range(sgd_params.epochs):
        start = time.time()
        for batch in make_batches(dataset, sgd_params.minibatch_size):
            average_grad = Update()
            average_loss = 0
            for sentence in batch:
                loss, grad = optimization_task.loss_and_gradient(sentence)
                average_grad.assign_madd(grad, 1)
                average_loss += loss
            average_grad.assign_mul(1 / sgd_params.minibatch_size)
            average_loss /= sgd_params.minibatch_size
            optimization_task.params().assign_madd(average_grad, -sgd_params.learning_rate)
            all_params_avg.assign_madd(optimization_task.params(), 1)
        after_each_epoch_fn()
        print(f"Epoch time: {time.time() - start} s")
    all_params_avg.assign_mul(1 / (sgd_params.epochs * int(len(dataset) / sgd_params.minibatch_size + 1)))
    if sgd_params.average:
        optimization_task.params().assign(all_params_avg)
    after_each_epoch_fn()

### Part VIII: Training loop

The train function combines everthing you used below to get new 

In [14]:
def train(
    tags_path='./data/tags',
    train_dataset='./data/en-ud-train.conllu',
    dev_dataset='./data/en-ud-dev.conllu',
    model_path='./model.npz',
    
    sgd_epochs=15,
    sgd_learning_rate=0.02,
    sgd_minibatch_size=32,
    sgd_average=True,
    
    # Number of context tags in output tagging to use for features
    tagger_src_window=2,
    
    # Number of context tags in output tagging to use for features
    tagger_dst_order=3,
    
    # Maximal number of prefix/suffix letters to use for features
    tagger_max_suffix=4,
    
    # Width for beam search (0 means unstructured)
    beam_size=1,
    
    # Parameter vector size (for hashing)
    nparams= 2 ** 22,
):
    """ Train a pos-tagger model and save it's parameters to :model: """

    # Beam size.
    optimization_task_cls = StructuredPerceptronOptimizationTask
    if beam_size == 0:
        beam_size = 1
        optimization_task_cls = UnstructuredPerceptronOptimizationTask

    # Parse cmdargs.
    tags = read_tags(tags_path)
    train_dataset = read_tagged_sentences(train_dataset)
    dev_dataset = read_tagged_sentences(dev_dataset)
    params = None
    if os.path.exists(model_path):
        params = pickle.load(open(model_path, 'rb'))
    sgd_params = SGDParams(
        epochs=sgd_epochs,
        learning_rate=sgd_learning_rate,
        minibatch_size=sgd_minibatch_size,
        average=sgd_average
        )
    tagger_params = TaggerParams(
        src_window=tagger_src_window,
        dst_order=tagger_dst_order,
        max_suffix=tagger_max_suffix,
        beam_size=beam_size,
        nparams=nparams
        )

    # Load optimization task
    optimization_task = optimization_task_cls(tagger_params, tags)
    if params is not None:
        print('\n\nLoading parameters from %s\n\n' % model_path)
        optimization_task.params().assign(params)

    # Validation.
    def after_each_epoch_fn():
        model = LinearModel(nparams)
        model.params().assign(optimization_task.params())
        tagged_sentences = tag_sentences(dev_dataset, model, tags, tagger_params)
        q = pprint.pformat(tagging_quality(out=tagged_sentences, ref=dev_dataset))
        print()
        print(q)
        print()

        # Save parameters.
        print('\n\nSaving parameters to %s\n\n' % model_path)
        pickle.dump(optimization_task.params(), open(model_path, 'wb'))

    # Run SGD.
    sgd(sgd_params, optimization_task, train_dataset, after_each_epoch_fn)


In [15]:
# train a model with default params
train(model_path='./default_model.npz', beam_size=0)



Loading parameters from ./default_model_6.npz


[TaggedWord(text='From', tag='NOUN'), TaggedWord(text='the', tag='NOUN'), TaggedWord(text='AP', tag='NOUN'), TaggedWord(text='comes', tag='NOUN'), TaggedWord(text='this', tag='NOUN'), TaggedWord(text='story', tag='NOUN'), TaggedWord(text=':', tag='NOUN')]

0.16280971461280563



Saving parameters to ./default_model_6.npz




KeyboardInterrupt: 

### Part IX: Evaluate the trained model

In [30]:
def test(
    tags='./data/tags',
    dataset='./data/en-ud-dev.conllu',
    model='./model.npz',
    
    # model and inference params; see train for their description
    tagger_src_window=2,
    tagger_dst_order=3,
    tagger_max_suffix=4,
    beam_size=1,
):


    tags = read_tags(tags)
    dataset = read_tagged_sentences(dataset)
    params = pickle.load(open(model, 'rb'))
    tagger_params = TaggerParams(
        src_window=tagger_src_window,
        dst_order=tagger_dst_order,
        max_suffix=tagger_max_suffix,
        beam_size=beam_size,
        nparams=0
        )

    # Load model.
    model = LinearModel(params.values.shape[0])
    model.params().assign(params)

    # Tag all sentences.
    tagged_sentences = tag_sentences(dataset, model, tags, tagger_params)

    # Write tagged sentences.
    for tagged_sentence in tagged_sentences:
        write_tagged_sentence(tagged_sentence, sys.stdout)

    # Measure and print quality.
    q = pprint.pformat(tagging_quality(out=tagged_sentences, ref=dataset))
    print(q, file=sys.stderr)


In [ ]:
# test 
test(model='./default_model.npz')

# sanity check: accuracy > 90%.

### Part X: play with it

_This part is optional_

Once you've built something, it's only natural to test the limits of your contraption.

At minumum, we want you to find out how default model accuracy depends on __beam size__

To get maximum points, your model should get final quality >= 93% 

Any further analysis is welcome, as always.

In [ ]:
<YOUR CODE>